In [1]:
!pip install opendatasets --upgrade --quiet

In [2]:
import opendatasets as od
dataset_url = 'https://www.kaggle.com/datasets/arnaud58/selfie2anime'
od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: h4hemant
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/arnaud58/selfie2anime


100%|██████████| 390M/390M [00:11<00:00, 35.9MB/s]


In [5]:
import os
data_dir = '/content/selfie2anime'
print(os.listdir(data_dir))

['images']


In [4]:
import shutil
checkpoints_dir = os.path.join(data_dir, '.ipynb_checkpoints')
if os.path.exists(checkpoints_dir):
    # Remove the folder
    shutil.rmtree(checkpoints_dir)
    print('.ipynb_checkpoints folder has been deleted.')
else:
    print('.ipynb_checkpoints folder not found.')

.ipynb_checkpoints folder has been deleted.


In [6]:
import os
from PIL import Image

input_folder = "/content/selfie2anime/images"
output_folder = "/content/selfie2anime/processed_anime_images"
resolution = 256

os.makedirs(output_folder, exist_ok=True)

for img_name in os.listdir(input_folder):
    img_path = os.path.join(input_folder, img_name)
    try:
        img = Image.open(img_path).convert("RGB")
        img = img.resize((resolution, resolution), Image.LANCZOS)
        img.save(os.path.join(output_folder, img_name))
    except Exception as e:
        print(f"Skipping {img_name}: {e}")


In [7]:
import pandas as pd

In [8]:
!git clone https://github.com/NVlabs/stylegan3.git

Cloning into 'stylegan3'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 212 (delta 99), reused 90 (delta 90), pack-reused 49 (from 1)
Receiving objects: 100% (212/212), 4.16 MiB | 24.21 MiB/s, done.
Resolving deltas: 100% (108/108), done.


In [9]:
%cd stylegan3

/content/stylegan3


In [10]:
!pip install torch torchvision numpy scipy tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 24.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [11]:
!mkdir outputs

In [12]:
!python dataset_tool.py --source=/content/selfie2anime/processed_anime_images --dest=anime_tfrecords --resolution=256x256

100% 3400/3400 [00:20<00:00, 169.57it/s]


In [13]:
!pip install ninja

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 13.1 MB/s eta 0:00:00


In [16]:
import torch
torch.cuda.empty_cache()

In [ ]:
!python train.py --outdir=/content/stylegan3/outputs \
                 --cfg=stylegan3-t \
                 --data=anime_tfrecords \
                 --gpus=1 \
                 --batch=8 \
                 --snap=10 \
                 --gamma=5 \
                 --kimg=5000 \
                 --metrics=fid50k_full


Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 32768,
    "channel_max": 512,
    "magnitude_ema_beta": 0.9997227795604651
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 5.0
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "prefetch_fact